In [1]:
import os
%pwd
os.chdir("../")
%pwd

'C:\\Users\\ahmad\\PycharmProjects\\ISE-Projekt'

In [2]:
import tensorflow as tf

In [5]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [23]:
from dataclasses import dataclass
from pathlib import Path

from Classification.constants import *
from Classification.utils.common import read_yaml, create_directories, save_json

from urllib.parse import urlparse

In [24]:
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [25]:
#from Classification.constants import *
#from Classification.utils.common import read_yaml, create_directories, save_json

In [26]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/brain_tumor_dataset",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [27]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    # Validoerung Generator
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
    #Load model:
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    # do the evaluation:
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    # write the score:
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

In [28]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e

[2024-01-01 23:02:30,299: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-01 23:02:30,302: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-01 23:02:30,303: INFO: common: created directory at: artifacts]
Found 75 images belonging to 2 classes.
5/5 [==============================] - 5s 1s/step - loss: 0.7309 - accuracy: 0.4267
[2024-01-01 23:02:36,354: INFO: common: json file saved at: scores.json]
